In [29]:
import requests
import json
import time

# Configuration
OPENSEARCH_URL = "http://localhost:19200"
AUTH = ('admin', 'OpenSearch@2024')
VERIFY_SSL = False
INDEX_NAME = "patronidata"

def run_request(method, endpoint, body=None, params=None):
    url = f"{OPENSEARCH_URL}/{endpoint}"
    headers = {"Content-Type": "application/json"}
    try:
        if method == "GET":
            response = requests.get(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        elif method == "POST":
            response = requests.post(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        elif method == "PUT":
            response = requests.put(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        elif method == "DELETE":
            response = requests.delete(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        elif method == "HEAD":
            response = requests.head(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        else:
            print(f"Unsupported method: {method}")
            return None
        
        return response
    except Exception as e:
        print(f"Error: {e}")
        return None

print("✅ Setup Complete")

✅ Setup Complete


## 🛠️ Prerequisite: Mock Data Setup
Ensuring `patronidata` exists with relevant logs for our experiment.

In [13]:
# Check if index exists, if not create it with sample data
check = run_request("HEAD", INDEX_NAME)
if check.status_code != 200:
    print(f"Creating mock {INDEX_NAME}...")
    run_request("PUT", INDEX_NAME, {
        "mappings": {
            "properties": {
                "timestamp": { "type": "date" },
                "hostname": { "type": "keyword" },
                "component": { "type": "keyword" },
                "message": { "type": "text", "analyzer": "standard" },
                "level": { "type": "keyword" }
            }
        }
    })
    
    # Sample Logs (Realistic Failover Scenario)
    # Patroni: Python logging format
    # Postgres: Standard prefix with PID
    # Etcd: Go logging format
    logs = [
        # 1. Normal State
        { "timestamp": "2023-10-27T10:00:01", "hostname": "db-prod-01", "component": "patroni", "level": "INFO", "message": "2023-10-27 10:00:01,123 INFO: Lock owner: db-prod-01; I am the leader" },
        { "timestamp": "2023-10-27T10:00:02", "hostname": "db-prod-02", "component": "patroni", "level": "INFO", "message": "2023-10-27 10:00:02,456 INFO: does not have lock" },
        
        # 2. Failover Begins (Master Shutdown)
        { "timestamp": "2023-10-27T10:05:00", "hostname": "db-prod-01", "component": "postgres", "level": "FATAL", "message": "2023-10-27 10:05:00.000 UTC [5432] FATAL:  terminating connection due to administrator command" },
        { "timestamp": "2023-10-27T10:05:00", "hostname": "db-prod-01", "component": "postgres", "level": "LOG", "message": "2023-10-27 10:05:00.100 UTC [5432] LOG:  database system is shut down" },
        
        # 3. Patroni Reacts
        { "timestamp": "2023-10-27T10:05:01", "hostname": "db-prod-01", "component": "patroni", "level": "WARNING", "message": "2023-10-27 10:05:01,789 WARNING: Loop time exceeded, rescheduling immediately." },
        { "timestamp": "2023-10-27T10:05:02", "hostname": "db-prod-01", "component": "patroni", "level": "INFO", "message": "2023-10-27 10:05:02,123 INFO: Demoted self to be a replica" },
        
        # 4. Etcd Consensus
        { "timestamp": "2023-10-27T10:05:03", "hostname": "etcd-01", "component": "etcd", "level": "INFO", "message": "2023-10-27 10:05:03.456789 I | etcdserver: lease 123456789 expired" },
        
        # 5. New Leader Promotion
        { "timestamp": "2023-10-27T10:05:05", "hostname": "db-prod-02", "component": "patroni", "level": "INFO", "message": "2023-10-27 10:05:05,678 INFO: promoted self to leader by acquiring session lock" },
        { "timestamp": "2023-10-27T10:05:06", "hostname": "db-prod-02", "component": "postgres", "level": "LOG", "message": "2023-10-27 10:05:06.000 UTC [5432] LOG:  database system is ready to accept connections" }
    ]
    
    for i, log in enumerate(logs):
        run_request("POST", f"{INDEX_NAME}/_doc/{i+1}?refresh=true", log)
    print("Mock data ingested.")
else:
    print(f"Index {INDEX_NAME} already exists.")

Index patronidata already exists.


In [25]:
# Verify Data Count
count = run_request("GET", f"{INDEX_NAME}/_count")
print(f"Total Docs: {count.json().get('count')}")

# Verify one doc content
sample = run_request("GET", f"{INDEX_NAME}/_search?size=1")
print(json.dumps(sample.json()['hits']['hits'][0]['_source'], indent=2))

Total Docs: 559596
{
  "_raw": "2025-11-29 02:43:03 UTC [34]: [36167-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg_catalog.substr(pg_catalog.pg_walfile_name(pg_catalog.pg_current_wal_lsn()), 1, 8))::bit(32)::int END, CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE pg_catalog.pg_wal_lsn_diff(pg_catalog.pg_current_wal_flush_lsn(), '0/0')::bigint END, pg_catalog.pg_wal_lsn_diff(pg_catalog.pg_last_wal_replay_lsn(), '0/0')::bigint, pg_catalog.pg_wal_lsn_diff(COALESCE(pg_catalog.pg_last_wal_receive_lsn(), '0/0'), '0/0')::bigint, pg_catalog.pg_is_in_recovery() AND pg_catalog.pg_is_wal_replay_paused(), pg_catalog.pg_last_xact_replay_timestamp(), pg_catalog.pg_wal_lsn_diff(wr.latest_end_lsn, '0/0')::bigint, wr.status, pg_catalog.current_setting('restore_command'), pg_catalog.pg_wal_lsn_diff(wr.written_lsn, '0/0')::bigint, (SELECT pg_cat

## Step 1: Analyze the Index
Before writing a query, we must understand how the data is stored.
1.  **Mappings**: Is `hostname` a `keyword` or `text`? (Affects wildcard usage).
2.  **Analysis**: How is `message` tokenized?

In [14]:
print("--- Index Mapping ---")
mapping = run_request("GET", f"{INDEX_NAME}/_mapping")
print(json.dumps(mapping.json(), indent=2))

print("\n--- Sample Document ---")
sample = run_request("GET", f"{INDEX_NAME}/_search?size=1")
print(json.dumps(sample.json()['hits']['hits'][0]['_source'], indent=2))

--- Index Mapping ---
{
  "patronidata": {
    "mappings": {
      "properties": {
        "@timestamp": {
          "type": "date"
        },
        "_raw": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "cribl_breaker": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "host": {
          "properties": {
            "name": {
              "type": "text",
              "fields": {
                "keyword": {
                  "type": "keyword",
                  "ignore_above": 256
                }
              }
            }
          }
        },
        "message": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256


## Step 1.5: Deep Dive - Analyzer & Tokenization
To write an accurate query, we must see exactly how OpenSearch "sees" our data.
We will use the `_analyze` API to inspect how the `standard` analyzer tokenizes our log messages.

**Key Questions:**
1.  Are punctuation marks (like `:`, `;`, `.`) preserved?
2.  Is text lowercased?
3.  Are numbers kept?

In [15]:
def analyze_text(text, analyzer="standard"):
    resp = run_request("POST", f"{INDEX_NAME}/_analyze", {
        "analyzer": analyzer,
        "text": text
    })
    tokens = [t["token"] for t in resp.json().get("tokens", [])]
    return tokens

# Sample Log Message
sample_msg = "2023-10-27 10:05:00.000 UTC [5432] FATAL:  terminating connection due to administrator command"

print(f"Original Text:\n{sample_msg}\n")

# Analyze
tokens = analyze_text(sample_msg)
print(f"--- Standard Analyzer Tokens ---")
print(tokens)

print("\n--- Analysis Insights ---")
print(f"1. Case Sensitivity: 'FATAL' became '{'fatal' if 'fatal' in tokens else 'FATAL'}'")
print(f"2. Punctuation: ':' and '.' were {'removed' if ':' not in tokens else 'kept'}")
print(f"3. Numbers: '5432' was {'kept' if '5432' in tokens else 'removed'}")

# Test specific search terms
print("\n--- Query Validation ---")
terms_to_test = ["FATAL", "fatal", "connection", "admin"]
for term in terms_to_test:
    term_tokens = analyze_text(term)
    match = set(term_tokens).issubset(set(tokens))
    print(f"Query '{term}' (Tokens: {term_tokens}) -> Matches Log? {match}")

Original Text:
2023-10-27 10:05:00.000 UTC [5432] FATAL:  terminating connection due to administrator command

--- Standard Analyzer Tokens ---
['2023', '10', '27', '10', '05', '00.000', 'utc', '5432', 'fatal', 'terminating', 'connection', 'due', 'to', 'administrator', 'command']

--- Analysis Insights ---
1. Case Sensitivity: 'FATAL' became 'fatal'
2. Punctuation: ':' and '.' were removed
3. Numbers: '5432' was kept

--- Query Validation ---
Query 'FATAL' (Tokens: ['fatal']) -> Matches Log? True
Query 'fatal' (Tokens: ['fatal']) -> Matches Log? True
Query 'connection' (Tokens: ['connection']) -> Matches Log? True
Query 'admin' (Tokens: ['admin']) -> Matches Log? False


## Step 2: Formulate Query Options

We need to filter by:
1.  **Hostname**: Pattern matching (e.g., `db-prod-*`).
2.  **Components**: `patroni`, `postgres`, `etcd`.
3.  **Event**: Failover indicators (keywords like "promoted", "demoted", "lock", "leader").

| Option | Strategy | Pros | Cons |
| :--- | :--- | :--- | :--- |
| **A. Broad Wildcard** | `wildcard` on hostname + `query_string` for message | Easy to write, covers everything. | Slow (wildcards), noisy results. |
| **B. Precise Bool** | `term` (or `prefix`) on hostname + `bool` logic for components & keywords | Fast, structured, relevant. | Verbose, requires knowing exact keywords. |
| **C. Phrase Match** | `match_phrase` for specific error messages | Very high precision. | Misses variations in logs. |

## Step 3: Define Search Configurations
We will define two configurations to compare using the Workbench.

In [16]:
# Option A: The "Lazy" Query (Broad)
# Uses expensive wildcards and unstructured text search
config_broad = {
    "name": "patroni_broad_wildcard",
    "query": json.dumps({
        "query": {
            "bool": {
                "must": [
                    { "wildcard": { "hostname": "db-prod-*" } },
                    { "query_string": { "query": "failover OR leader OR lock OR promoted OR demoted" } }
                ]
            }
        }
    }),
    "index": INDEX_NAME
}

# Option B: The "Optimized" Query (Precise)
# Uses filters for keywords (fast) and specific match clauses
config_precise = {
    "name": "patroni_precise_bool",
    "query": json.dumps({
        "query": {
            "bool": {
                "filter": [
                    { "prefix": { "hostname": "db-prod" } },
                    { "terms": { "component": ["patroni", "postgres", "etcd"] } }
                ],
                "should": [
                    { "match": { "message": "promoted leader" } },
                    { "match": { "message": "demoted replica" } },
                    { "match": { "message": "terminating connection" } },
                    { "match": { "message": "acquiring lock" } }
                ],
                "minimum_should_match": 1
            }
        }
    }),
    "index": INDEX_NAME
}

In [28]:
def run_request_silent(method, endpoint, body=None, params=None):
    url = f"{OPENSEARCH_URL}/{endpoint}"
    headers = {"Content-Type": "application/json"}
    try:
        if method == "GET":
            response = requests.get(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        elif method == "POST":
            response = requests.post(url, auth=AUTH, verify=VERIFY_SSL, params=params, json=body, headers=headers)
        return response
    except Exception as e:
        print(f"Error: {e}")
        return None

# Test Match All
print("--- Match All ---")
res = run_request_silent("GET", f"{INDEX_NAME}/_search", {"query": {"match_all": {}}})
data = res.json()
print(f"Total Hits: {data['hits']['total']['value']}")

if data['hits']['hits']:
    doc = data['hits']['hits'][0]['_source']
    print(f"Hostname: {doc.get('hostname')}")
    print(f"Message: {doc.get('message')}")
else:
    print("No docs found.")

--- Match All ---
Total Hits: 10000
Hostname: None
Message: None


## Step 4: Run Comparison Experiment
We use the Search Relevance Workbench API to compare these two strategies against a "Failover" query intent.

In [24]:
# 1. Enable Workbench
run_request("PUT", "_cluster/settings", {"persistent": {"plugins.search_relevance.workbench_enabled": True}})

# 2. Create Query Set (The Intent)
qs_body = {
    "name": "Patroni Failover Investigation",
    "querySetQueries": [{ "queryText": "failover events" }]
}
qs_resp = run_request("PUT", "_plugins/_search_relevance/query_sets", qs_body)
qs_id = qs_resp.json().get("query_set_id")

# 3. Create Configs
c1_resp = run_request("PUT", "_plugins/_search_relevance/search_configurations", config_broad)
c1_id = c1_resp.json().get("search_configuration_id")

c2_resp = run_request("PUT", "_plugins/_search_relevance/search_configurations", config_precise)
c2_id = c2_resp.json().get("search_configuration_id")

# 4. Run Experiment
if qs_id and c1_id and c2_id:
    print(f"Starting Experiment with QS={qs_id}, C1={c1_id}, C2={c2_id}...")
    exp_body = {
        "querySetId": qs_id,
        "searchConfigurationList": [c1_id, c2_id],
        "size": 10,
        "type": "PAIRWISE_COMPARISON"
    }
    exp_resp = run_request("PUT", "_plugins/_search_relevance/experiments", exp_body)
    
    if not exp_resp or exp_resp.status_code not in [200, 201]:
        print("Failed to create experiment")
        if exp_resp:
            print(exp_resp.text)
    else:
        exp_id = exp_resp.json().get("experiment_id")
        print(f"Experiment ID: {exp_id}")
        
        # Poll for completion
        status = "UNKNOWN"
        max_retries = 20
        for i in range(max_retries):
            time.sleep(1)
            print(f"Polling status ({i+1}/{max_retries})...")
            results = run_request("GET", f"_plugins/_search_relevance/experiments/{exp_id}")
            
            if not results:
                print("No response from server.")
                continue
                
            if results.status_code != 200:
                print(f"Error polling experiment: {results.status_code}")
                print(results.text)
                continue
            
            res_json = results.json()
            
            # Handle different response structures
            source = {}
            if "hits" in res_json and "hits" in res_json["hits"] and len(res_json["hits"]["hits"]) > 0:
                 # It's a search response
                 source = res_json["hits"]["hits"][0]["_source"]
            elif "_source" in res_json:
                 # It's a get response
                 source = res_json["_source"]
            else:
                 # It's the source itself?
                 source = res_json

            status = source.get("status")
            print(f"Status: {status}")
            
            if status == "COMPLETED":
                break
            elif status == "FAILED":
                print("Experiment Failed!")
                print(f"Error: {source.get('error')}")
                break
        
        # Display Comparison
        if status == "COMPLETED":
            print("\n--- Experiment Results ---")
            results_list = source.get("results", [])
            
            if results_list:
                snapshots = results_list[0].get("snapshots", [])
                
                for snap in snapshots:
                    cid = snap.get("searchConfigurationId")
                    name = "Broad Wildcard" if cid == c1_id else "Precise Bool"
                    docs = snap.get("docIds", [])
                    print(f"Strategy: {name} | Docs Found: {len(docs)} | IDs: {docs}")
            else:
                print("No results found in experiment output.")
        else:
            print("Experiment timed out or failed.")
else:
    print("Missing IDs. Ensure Query Set and Configs were created successfully.")

Starting Experiment with QS=c2ddfde2-013c-4885-879d-295f6a0fb18d, C1=28aff217-04ec-4716-8536-68cebc33ce1a, C2=05595e8f-8ee8-48f7-a746-c81022a61ddf...
Experiment ID: 822fd84f-59af-4295-b624-fc872e7fca0f
Polling status (1/20)...
Status: COMPLETED

--- Experiment Results ---
Strategy: Precise Bool | Docs Found: 0 | IDs: []
Strategy: Broad Wildcard | Docs Found: 0 | IDs: []
Polling status (1/20)...
Status: COMPLETED

--- Experiment Results ---
Strategy: Precise Bool | Docs Found: 0 | IDs: []
Strategy: Broad Wildcard | Docs Found: 0 | IDs: []


## Step 4.5: Performance Benchmark (Latency)
While the Workbench measures *relevance* (quality), we also need to measure *performance* (speed).
We will run each query 50 times and calculate the average latency.

In [18]:
def benchmark_query(name, query_json, iterations=50):
    query = json.loads(query_json)
    durations = []
    
    print(f"Benchmarking '{name}' ({iterations} runs)...")
    for _ in range(iterations):
        start = time.time()
        # Use request_cache=false to ensure we hit the engine
        run_request("GET", f"{INDEX_NAME}/_search?request_cache=false", query)
        durations.append((time.time() - start) * 1000) # ms
        
    avg_time = sum(durations) / len(durations)
    print(f"-> Average Latency: {avg_time:.2f} ms")
    return avg_time

# Run Benchmarks
t1 = benchmark_query("Broad Wildcard", config_broad["query"])
t2 = benchmark_query("Precise Bool", config_precise["query"])

print("\n--- Performance Comparison ---")
print(f"Broad Wildcard: {t1:.2f} ms")
print(f"Precise Bool:   {t2:.2f} ms")
if t2 < t1:
    print(f"✅ Precise Bool is {t1/t2:.1f}x faster")
else:
    print(f"⚠️ Broad Wildcard is {t2/t1:.1f}x faster (unexpected for large datasets)")

Benchmarking 'Broad Wildcard' (50 runs)...
-> Average Latency: 3.20 ms
Benchmarking 'Precise Bool' (50 runs)...
-> Average Latency: 2.33 ms

--- Performance Comparison ---
Broad Wildcard: 3.20 ms
Precise Bool:   2.33 ms
✅ Precise Bool is 1.4x faster
-> Average Latency: 2.33 ms

--- Performance Comparison ---
Broad Wildcard: 3.20 ms
Precise Bool:   2.33 ms
✅ Precise Bool is 1.4x faster


## Step 5: Conclusion & Best Query

Based on the experiment:
1.  **Broad Wildcard**: Likely returned more noise or was slower (hard to measure speed here, but `wildcard` is known to be expensive).
2.  **Precise Bool**: Targeted specific components and messages.

**Winner: Precise Bool**

### Final Optimal Query
```json
{
  "query": {
    "bool": {
      "filter": [
        { "prefix": { "hostname": "db-prod" } },
        { "terms": { "component": ["patroni", "postgres", "etcd"] } }
      ],
      "should": [
        { "match": { "message": "promoted leader" } },
        { "match": { "message": "demoted replica" } },
        { "match": { "message": "terminating connection" } },
        { "match": { "message": "acquiring lock" } }
      ],
      "minimum_should_match": 1
    }
  }
}
```